In [33]:
from __future__ import print_function
import os
import h5py
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR, StepLR,ReduceLROnPlateau
from torch.utils.data import Dataset,DataLoader
import torch.nn.init as init

import sklearn.metrics as metrics

In [34]:
#wandb used to log metrics and results
import wandb
#insert here wandb key
WANDB_KEY=""
if WANDB_KEY:
    wandb.login(key=WANDB_KEY)

## Dataset

In [35]:
#translate each pointcloud of a random value
def translate_pointcloud(pointcloud):
    xyz1 = np.random.uniform(low=2./3., high=3./2., size=[3])
    xyz2 = np.random.uniform(low=-0.2, high=0.2, size=[3])
       
    translated_pointcloud = np.add(np.multiply(pointcloud, xyz1), xyz2).astype('float32')
    return translated_pointcloud

#create dataset with fragments
class MyDataset(Dataset):
    def __init__(self,args,extra_args,partition='train'):
        self.args=args
        self.dataset_file=h5py.File(extra_args["dataset_file_path"], 'r')
        self.num_train=min(args["num_train_samples"],len(self.dataset_file["train_data"]))
        self.num_test=min(args["num_test_samples"],len(self.dataset_file["test_data"]))
        self.num_val=min(args["num_test_samples"],len(self.dataset_file["val_data"]))
        
        #get datasets
        self.train_set=self.dataset_file["train_data"][:self.num_train]
        self.train_labels=self.dataset_file["train_labels"][:self.num_train]
        self.train_normals=self.dataset_file["train_normals"][:self.num_train]
        
        self.test_set=self.dataset_file["test_data"][:self.num_test]
        self.test_normals=self.dataset_file["test_normals"][:self.num_test]
        self.test_labels=self.dataset_file["test_labels"][:self.num_test]

        self.val_set=self.dataset_file["val_data"][:self.num_val]
        self.val_normals=self.dataset_file["val_normals"][:self.num_val]
        self.val_labels=self.dataset_file["val_labels"][:self.num_val]


        self.partition = partition        

    def __getitem__(self, item):
        if self.partition=="train":
            pointclouds_pair=self.train_set[item]
            normals_pair=self.train_normals[item]
            label=self.train_labels[item]
            
            #preprocess pointclouds
            pointcloud1=pointclouds_pair[0,:,:]
            pointcloud2=pointclouds_pair[1,:,:]
            pointcloud1 = translate_pointcloud(pointcloud1)
            pointcloud2 = translate_pointcloud(pointcloud2)
            pointclouds_pair=torch.Tensor([pointcloud1,pointcloud2])

        elif self.partition=="test":        
            pointclouds_pair=self.test_set[item]
            normals_pair=self.test_normals[item]
            label=self.test_labels[item]
        
        elif self.partition=="val":        
            pointclouds_pair=self.val_set[item]
            normals_pair=self.val_normals[item]
            label=self.val_labels[item]

        return pointclouds_pair,normals_pair, label

    def __len__(self):
        if self.partition=="train":
            return self.num_train
        elif self.partition=="val":
            return self.num_val
        else:
            return self.num_test


## Util

In [36]:

def cal_loss(pred, gold, smoothing=False):
    ''' Calculate cross entropy loss, apply label smoothing if needed. '''
    #we do not use label smoothing

    gold = gold.contiguous().view(-1)

    if smoothing:
        eps = 0.2
        n_class = pred.size(1)

        one_hot = torch.zeros_like(pred).scatter(1, gold.view(-1, 1), 1)
        one_hot = one_hot * (1 - eps) + (1 - one_hot) * eps / (n_class - 1)
        log_prb = F.log_softmax(pred, dim=1)

        loss = -(one_hot * log_prb).sum(dim=1).mean()
    else:
        loss = F.binary_cross_entropy(pred, gold, reduction='mean')

    return loss

#class to print on file some debug stuff
class IOStream():
    def __init__(self, path):
        self.f = open(path, 'a')

    def cprint(self, text):
        print(text)
        self.f.write(text+'\n')
        self.f.flush()

    def close(self):
        self.f.close()


## Models

In [37]:
#zero out some points in the pointcluod and related normal
class PointDropout(torch.nn.Module):

    def __init__(self, p=0.0):
        super().__init__()
        self.p = torch.nn.Parameter(torch.tensor(1 - p), requires_grad=False)
        self.distrib = torch.distributions.Bernoulli(self.p)

    def forward(self, x,n):
        #dropout works only in training
        if self.training:
            dropout_mask = self.distrib.sample(torch.Size([x.shape[0],x.shape[2]]))
            for i in range(len(x)):
                x[i]=x[i]*dropout_mask[i]
                n[i]=n[i]*dropout_mask[i]
        return x,n

In [38]:
#k-nearest neighbour, part of edge convolution step
def knn(x, k):
    inner = -2*torch.matmul(x.transpose(2, 1), x)
    xx = torch.sum(x**2, dim=1, keepdim=True)
    pairwise_distance = -xx - inner - xx.transpose(2, 1)
 
    idx = pairwise_distance.topk(k=k, dim=-1)[1]   # (batch_size, num_points, k)
    return idx


def get_graph_feature(x, k=20,device="cpu", idx=None, dim9=False):
    batch_size = x.size(0)
    num_points = x.size(2)

    x = x.view(batch_size, -1, num_points)

    if idx is None:
        if dim9 == False:
            idx = knn(x, k=k)   # (batch_size, num_points, k)
        else:
            idx = knn(x[:, 6:], k=k)
    device = torch.device(device)

    idx_base = torch.arange(0, batch_size, device=device).view(-1, 1, 1)*num_points

    idx = idx + idx_base

    idx = idx.view(-1)
 
    _, num_dims, _ = x.size()

    x = x.transpose(2, 1).contiguous()   # (batch_size, num_points, num_dims)  -> (batch_size*num_points, num_dims) #   batch_size * num_points * k + range(0, batch_size*num_points)
    feature = x.view(batch_size*num_points, -1)[idx, :]
    feature = feature.view(batch_size, num_points, k, num_dims) 
    x = x.view(batch_size, num_points, 1, num_dims).repeat(1, 1, k, 1)
    
    feature = torch.cat((feature-x, x), dim=3).permute(0, 3, 1, 2).contiguous()
  
    return feature      # (batch_size, 2*num_dims, num_points, k)


### Model 1
Normals are concatenated to the output of the two DGCNNs and then 3 classification layers act as classifier

In [39]:

class DGCNN_cls2(nn.Module):
    def __init__(self, args, output_channels=1):
        super(DGCNN_cls2, self).__init__()
        self.args = args
        self.device="cuda" if self.args["cuda"] else "cpu"
        self.k = args["k"]
        
        self.bn1 = nn.BatchNorm2d(64)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm2d(128)
        self.bn4 = nn.BatchNorm2d(256)
        self.bn5 = nn.BatchNorm1d(args["emb_dims"])

        self.conv1 = nn.Sequential(nn.Conv2d(6, 64, kernel_size=1, bias=False),
                                   self.bn1,
                                   nn.LeakyReLU(negative_slope=0.2))
        self.conv2 = nn.Sequential(nn.Conv2d(64*2, 64, kernel_size=1, bias=False),
                                   self.bn2,
                                   nn.LeakyReLU(negative_slope=0.2))
        self.conv3 = nn.Sequential(nn.Conv2d(64*2, 128, kernel_size=1, bias=False),
                                   self.bn3,
                                   nn.LeakyReLU(negative_slope=0.2))
        self.conv4 = nn.Sequential(nn.Conv2d(128*2, 256, kernel_size=1, bias=False),
                                   self.bn4,
                                   nn.LeakyReLU(negative_slope=0.2))
        self.conv5 = nn.Sequential(nn.Conv1d(512, args["emb_dims"], kernel_size=1, bias=False),
                                   self.bn5,
                                   nn.LeakyReLU(negative_slope=0.2))
        
        self.node_num=args["emb_dims"]
        dim_normal=args["num_points"]*3*2
        
        self.linear1 = nn.Linear(args["emb_dims"]*2 + dim_normal//2, self.node_num, bias=False)
        self.bn6 = nn.BatchNorm1d(self.node_num)
        self.dp1 = nn.Dropout(p=args["dropout"])
        
       
        self.dp3=nn.Dropout(p=args["dropout"])
        
        dim_normal=args["num_points"]*3
        
        self.linearAlternative = nn.Linear(args["emb_dims"]*4 + dim_normal*2, args["emb_dims"], bias=False)
        self.bnAlt = nn.BatchNorm1d(args["emb_dims"])
        self.linearAlternative2 = nn.Linear(args["emb_dims"], args["emb_dims"]//2, bias=False)
        self.bnAlt2 = nn.BatchNorm1d(args["emb_dims"]//2)
        self.linearAlternative3 = nn.Linear(args["emb_dims"]//2, output_channels, bias=False)
       
       
    
    def dgcnn_step(self,x):
        batch_size = x.size(0)
        x = get_graph_feature(x, k=self.k,device=self.device)           # (batch_size, 3, num_points) -> (batch_size, 3*2, num_points, k)
        x = self.conv1(x)                                               # (batch_size, 3*2, num_points, k) -> (batch_size, 64, num_points, k)
        x1 = x.max(dim=-1, keepdim=False)[0]                            # (batch_size, 64, num_points, k) -> (batch_size, 64, num_points)

        x = get_graph_feature(x1, k=self.k,device=self.device)          # (batch_size, 64, num_points) -> (batch_size, 64*2, num_points, k)
        x = self.conv2(x)                                               # (batch_size, 64*2, num_points, k) -> (batch_size, 64, num_points, k)
        x2 = x.max(dim=-1, keepdim=False)[0]                            # (batch_size, 64, num_points, k) -> (batch_size, 64, num_points)

        x = get_graph_feature(x2, k=self.k,device=self.device)          # (batch_size, 64, num_points) -> (batch_size, 64*2, num_points, k)
        x = self.conv3(x)                                               # (batch_size, 64*2, num_points, k) -> (batch_size, 128, num_points, k)
        x3 = x.max(dim=-1, keepdim=False)[0]                            # (batch_size, 128, num_points, k) -> (batch_size, 128, num_points)

        x = get_graph_feature(x3, k=self.k,device=self.device)          # (batch_size, 128, num_points) -> (batch_size, 128*2, num_points, k)
        x = self.conv4(x)                                               # (batch_size, 128*2, num_points, k) -> (batch_size, 256, num_points, k)
        x4 = x.max(dim=-1, keepdim=False)[0]                            # (batch_size, 256, num_points, k) -> (batch_size, 256, num_points)

        x = torch.cat((x1, x2, x3, x4), dim=1)                          # (batch_size, 64+64+128+256, num_points)

        x = self.conv5(x)                                               # (batch_size, 64+64+128+256, num_points) -> (batch_size, emb_dims, num_points)
        x1 = F.adaptive_max_pool1d(x, 1).view(batch_size, -1)           # (batch_size, emb_dims, num_points) -> (batch_size, emb_dims)
        x2 = F.adaptive_avg_pool1d(x, 1).view(batch_size, -1)           # (batch_size, emb_dims, num_points) -> (batch_size, emb_dims)
        x = torch.cat((x1, x2), 1)                                      # (batch_size, emb_dims*2)
        
        return x

    def forward(self, x,n,invert=False):
       
        if invert:
            first=x[:,1,:,:]
            second=x[:,0,:,:]
            second_n=n[:,0,:,:]
            first_n=n[:,1,:,:]
        else:
            first=x[:,0,:,:]
            first_n=n[:,0,:,:]
            second=x[:,1,:,:]
            second_n=n[:,1,:,:]
        
        
        #compute DGCNN output from both pointclouds 
        ret1=self.dgcnn_step(first)      
        ret2=self.dgcnn_step(second)
        
        normals=torch.cat((first_n,second_n),dim=1)
        #combine output and use classifier
        x=torch.cat((ret1,ret2),dim=1)
        #combine output and normals
        x=torch.cat((x,normals),dim=1)
        

        #classifier
        x=F.leaky_relu(self.bnAlt(self.linearAlternative(x)), negative_slope=0.2)
        x=self.dp3(x)
        x=F.leaky_relu(self.bnAlt2(self.linearAlternative2(x)), negative_slope=0.2)
        x=self.dp3(x)
        x=self.linearAlternative3(x)
        
        

        return x


### Model 2
Normals are concatenated to the output of the DGCNN of a single pointcloud and passed through a linear layer before being concatenated to the other pointcloud and normal; then 2 final classification layers act as classifier. This is the model that returns the best performances overall.

In [40]:

class DGCNN_cls(nn.Module):
    def __init__(self, args, output_channels=1):
        super(DGCNN_cls, self).__init__()
        self.args = args
        self.device="cuda" if self.args["cuda"] else "cpu"
        self.k = args["k"]
        
        self.bn1 = nn.BatchNorm2d(64)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm2d(128)
        self.bn4 = nn.BatchNorm2d(256)
        self.bn5 = nn.BatchNorm1d(args["emb_dims"])

        self.conv1 = nn.Sequential(nn.Conv2d(6, 64, kernel_size=1, bias=False),
                                   self.bn1,
                                   nn.LeakyReLU(negative_slope=0.2))
        self.conv2 = nn.Sequential(nn.Conv2d(64*2, 64, kernel_size=1, bias=False),
                                   self.bn2,
                                   nn.LeakyReLU(negative_slope=0.2))
        self.conv3 = nn.Sequential(nn.Conv2d(64*2, 128, kernel_size=1, bias=False),
                                   self.bn3,
                                   nn.LeakyReLU(negative_slope=0.2))
        self.conv4 = nn.Sequential(nn.Conv2d(128*2, 256, kernel_size=1, bias=False),
                                   self.bn4,
                                   nn.LeakyReLU(negative_slope=0.2))
        self.conv5 = nn.Sequential(nn.Conv1d(512, args["emb_dims"], kernel_size=1, bias=False),
                                   self.bn5,
                                   nn.LeakyReLU(negative_slope=0.2))
        
        self.node_num=args["final_dim_DGCNN"]
        dim_normal=args["num_points"]*3
        
        self.linear1 = nn.Linear(args["emb_dims"]*2+dim_normal, self.node_num, bias=False)
        self.bn6 = nn.BatchNorm1d(self.node_num)

        self.dp1 = nn.Dropout(p=args["dropout"])
        self.dp3=nn.Dropout(p=args["dropout"])
        self.pointDropout=PointDropout(args["point_dropout"])
        
        #classification layers
        self.linearAlternative = nn.Linear(self.node_num*2, args["dim_classification"], bias=False)
        self.bnAlt = nn.BatchNorm1d(args["dim_classification"])
        self.linearAlternative3 = nn.Linear(args["dim_classification"], output_channels, bias=False)
       

    
    def dgcnn_step(self,x,n):
        batch_size = x.size(0)
        x = get_graph_feature(x, k=self.k,device=self.device)           # (batch_size, 3, num_points) -> (batch_size, 3*2, num_points, k)
        x = self.conv1(x)                                               # (batch_size, 3*2, num_points, k) -> (batch_size, 64, num_points, k)
        x1 = x.max(dim=-1, keepdim=False)[0]                            # (batch_size, 64, num_points, k) -> (batch_size, 64, num_points)

        x = get_graph_feature(x1, k=self.k,device=self.device)          # (batch_size, 64, num_points) -> (batch_size, 64*2, num_points, k)
        x = self.conv2(x)                                               # (batch_size, 64*2, num_points, k) -> (batch_size, 64, num_points, k)
        x2 = x.max(dim=-1, keepdim=False)[0]                            # (batch_size, 64, num_points, k) -> (batch_size, 64, num_points)

        x = get_graph_feature(x2, k=self.k,device=self.device)          # (batch_size, 64, num_points) -> (batch_size, 64*2, num_points, k)
        x = self.conv3(x)                                               # (batch_size, 64*2, num_points, k) -> (batch_size, 128, num_points, k)
        x3 = x.max(dim=-1, keepdim=False)[0]                            # (batch_size, 128, num_points, k) -> (batch_size, 128, num_points)

        x = get_graph_feature(x3, k=self.k,device=self.device)          # (batch_size, 128, num_points) -> (batch_size, 128*2, num_points, k)
        x = self.conv4(x)                                               # (batch_size, 128*2, num_points, k) -> (batch_size, 256, num_points, k)
        x4 = x.max(dim=-1, keepdim=False)[0]                            # (batch_size, 256, num_points, k) -> (batch_size, 256, num_points)

        x = torch.cat((x1, x2, x3, x4), dim=1)                          # (batch_size, 64+64+128+256, num_points)

        x = self.conv5(x)                                               # (batch_size, 64+64+128+256, num_points) -> (batch_size, emb_dims, num_points)
        x1 = F.adaptive_max_pool1d(x, 1).view(batch_size, -1)           # (batch_size, emb_dims, num_points) -> (batch_size, emb_dims)
        x2 = F.adaptive_avg_pool1d(x, 1).view(batch_size, -1)           # (batch_size, emb_dims, num_points) -> (batch_size, emb_dims)
        x = torch.cat((x1, x2), 1)                                      # (batch_size, emb_dims*2)
        
        n=torch.flatten(n,start_dim=1)
        x=torch.cat((x,n),dim=1)
        x = F.leaky_relu(self.bn6(self.linear1(x))) # output is (batch_size, final_dim_DGCNN)
        x = self.dp1(x)
        return x

    def forward(self, x,n,invert=False):
       
        if invert:
            #use inverted pair
            first=x[:,1,:,:]
            second=x[:,0,:,:]
            second_n=n[:,0,:,:]
            first_n=n[:,1,:,:]
        else:
            first=x[:,0,:,:]
            first_n=n[:,0,:,:]
            second=x[:,1,:,:]
            second_n=n[:,1,:,:]
        

        first,first_n=self.pointDropout(first,first_n)
        second,second_n=self.pointDropout(second,second_n)

        #compute DGCNN output from both pointclouds 
        ret1=self.dgcnn_step(first,first_n)      
        ret2=self.dgcnn_step(second,second_n)

        #combine output and use classifier
        x=torch.cat((ret1,ret2),dim=1)
        x=F.leaky_relu(self.bnAlt(self.linearAlternative(x)))
        x=self.dp3(x)
        x=self.linearAlternative3(x)
        

        return x


### Alternative models without normals

#### Model 3 (without normals)


In [41]:
class DGCNN_cls_no_normals(nn.Module):
    def __init__(self, args, output_channels=1):
        super(DGCNN_cls_no_normals, self).__init__()
        self.args = args
        self.device="cuda" if self.args["cuda"] else "cpu"
        self.k = args["k"]
        
        self.bn1 = nn.BatchNorm2d(64)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm2d(128)
        self.bn4 = nn.BatchNorm2d(256)
        self.bn5 = nn.BatchNorm1d(args["emb_dims"])

        self.conv1 = nn.Sequential(nn.Conv2d(6, 64, kernel_size=1, bias=False),
                                   self.bn1,
                                   nn.LeakyReLU(negative_slope=0.2))
        self.conv2 = nn.Sequential(nn.Conv2d(64*2, 64, kernel_size=1, bias=False),
                                   self.bn2,
                                   nn.LeakyReLU(negative_slope=0.2))
        self.conv3 = nn.Sequential(nn.Conv2d(64*2, 128, kernel_size=1, bias=False),
                                   self.bn3,
                                   nn.LeakyReLU(negative_slope=0.2))
        self.conv4 = nn.Sequential(nn.Conv2d(128*2, 256, kernel_size=1, bias=False),
                                   self.bn4,
                                   nn.LeakyReLU(negative_slope=0.2))
        self.conv5 = nn.Sequential(nn.Conv1d(512, args["emb_dims"], kernel_size=1, bias=False),
                                   self.bn5,
                                   nn.LeakyReLU(negative_slope=0.2))
        self.linear1 = nn.Linear(args["emb_dims"]*2, 512, bias=False)
        self.bn6 = nn.BatchNorm1d(512)
        self.dp1 = nn.Dropout(p=args["dropout"])
        self.linear2 = nn.Linear(512, 256)
        self.bn7 = nn.BatchNorm1d(256)
        self.dp2 = nn.Dropout(p=args["dropout"])
        
        self.linear3 = nn.Linear(256, args["final_dim_DGCNN"])
        self.bn8 = nn.BatchNorm1d( args["final_dim_DGCNN"])
        self.dp3=nn.Dropout(p=args["dropout"])
       

        #final classification layer
        if args["dim_classification"]>0:
            self.beginClassification = nn.Linear(args["final_dim_DGCNN"]*2, args["dim_classification"])
            self.bn9 = nn.BatchNorm1d(  args["dim_classification"] )
            self.midClassification = nn.Linear(args["dim_classification"], args["dim_classification"])
            self.bn10 = nn.BatchNorm1d(  args["dim_classification"] )
            self.endClassification = nn.Linear(args["dim_classification"] , output_channels)

        else:
            self.classification= nn.Linear(args["final_dim_DGCNN"]*2, output_channels)


    
    def dgcnn_step(self,x):
        batch_size = x.size(0)
        x = get_graph_feature(x, k=self.k,device=self.device)           # (batch_size, 3, num_points) -> (batch_size, 3*2, num_points, k)
        x = self.conv1(x)                                               # (batch_size, 3*2, num_points, k) -> (batch_size, 64, num_points, k)
        x1 = x.max(dim=-1, keepdim=False)[0]                            # (batch_size, 64, num_points, k) -> (batch_size, 64, num_points)

        x = get_graph_feature(x1, k=self.k,device=self.device)          # (batch_size, 64, num_points) -> (batch_size, 64*2, num_points, k)
        x = self.conv2(x)                                               # (batch_size, 64*2, num_points, k) -> (batch_size, 64, num_points, k)
        x2 = x.max(dim=-1, keepdim=False)[0]                            # (batch_size, 64, num_points, k) -> (batch_size, 64, num_points)

        x = get_graph_feature(x2, k=self.k,device=self.device)          # (batch_size, 64, num_points) -> (batch_size, 64*2, num_points, k)
        x = self.conv3(x)                                               # (batch_size, 64*2, num_points, k) -> (batch_size, 128, num_points, k)
        x3 = x.max(dim=-1, keepdim=False)[0]                            # (batch_size, 128, num_points, k) -> (batch_size, 128, num_points)

        x = get_graph_feature(x3, k=self.k,device=self.device)          # (batch_size, 128, num_points) -> (batch_size, 128*2, num_points, k)
        x = self.conv4(x)                                               # (batch_size, 128*2, num_points, k) -> (batch_size, 256, num_points, k)
        x4 = x.max(dim=-1, keepdim=False)[0]                            # (batch_size, 256, num_points, k) -> (batch_size, 256, num_points)

        x = torch.cat((x1, x2, x3, x4), dim=1)                          # (batch_size, 64+64+128+256, num_points)

        x = self.conv5(x)                                               # (batch_size, 64+64+128+256, num_points) -> (batch_size, emb_dims, num_points)
        x1 = F.adaptive_max_pool1d(x, 1).view(batch_size, -1)           # (batch_size, emb_dims, num_points) -> (batch_size, emb_dims)
        x2 = F.adaptive_avg_pool1d(x, 1).view(batch_size, -1)           # (batch_size, emb_dims, num_points) -> (batch_size, emb_dims)
        x = torch.cat((x1, x2), 1)                                      # (batch_size, emb_dims*2)

        x = F.leaky_relu(self.bn6(self.linear1(x)), negative_slope=0.2) # (batch_size, emb_dims*2) -> (batch_size, 512)
        x = self.dp1(x)
        x = F.leaky_relu(self.bn7(self.linear2(x)), negative_slope=0.2) # (batch_size, 512) -> (batch_size, 256)
        x = self.dp2(x)
        x =  F.leaky_relu(self.bn8(self.linear3(x)), negative_slope=0.2) # (batch_size, 256) -> (batch_size, output_channels)
        x=self.dp3(x)
        return x

    def forward(self, x,n,invert=False):
        if invert:
            first=x[:,1,:,:]
            second=x[:,0,:,:]
        else:
            first=x[:,0,:,:]
            second=x[:,1,:,:]

        #compute DGCNN output from both pointclouds 
        ret1=self.dgcnn_step(first)      
        ret2=self.dgcnn_step(second)

        #combine output and use classifier
        x=torch.cat((ret1,ret2),dim=1)
        if self.args["dim_classification"]>0:
            x=F.leaky_relu(self.bn9(self.beginClassification(x)), negative_slope=0.2)
            x=self.dp3(x)
            x=F.leaky_relu(self.bn10(self.midClassification(x)), negative_slope=0.2)
            x=self.dp3(x)
            x=self.endClassification(x)
        else: 
            x=self.classification(x)

        return x


#### Model 4 (without normals)
Thsi model doesn't use the normals and the dgcnn net acts only as a feature extractor, it doesn't use linear layers.
The linear layers are only used after with both pointcloud features and act as classifier.

In [42]:
class DGCNN_cls2_no_normals(nn.Module):
    def __init__(self, args, output_channels=1):
        super(DGCNN_cls2_no_normals, self).__init__()
        self.args = args
        self.device="cuda" if self.args["cuda"] else "cpu"
        self.k = args["k"]
        
        self.bn1 = nn.BatchNorm2d(64)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm2d(128)
        self.bn4 = nn.BatchNorm2d(256)
        self.bn5 = nn.BatchNorm1d(args["emb_dims"])

        self.conv1 = nn.Sequential(nn.Conv2d(6, 64, kernel_size=1, bias=False),
                                   self.bn1,
                                   nn.LeakyReLU(negative_slope=0.2))
        self.conv2 = nn.Sequential(nn.Conv2d(64*2, 64, kernel_size=1, bias=False),
                                   self.bn2,
                                   nn.LeakyReLU(negative_slope=0.2))
        self.conv3 = nn.Sequential(nn.Conv2d(64*2, 128, kernel_size=1, bias=False),
                                   self.bn3,
                                   nn.LeakyReLU(negative_slope=0.2))
        self.conv4 = nn.Sequential(nn.Conv2d(128*2, 256, kernel_size=1, bias=False),
                                   self.bn4,
                                   nn.LeakyReLU(negative_slope=0.2))
        self.conv5 = nn.Sequential(nn.Conv1d(512, args["emb_dims"], kernel_size=1, bias=False),
                                   self.bn5,
                                   nn.LeakyReLU(negative_slope=0.2))
        self.dp3=nn.Dropout(p=args["dropout"])

        #final classification layer
        if args["dim_classification"]>0:
            self.beginClassification = nn.Linear(args["emb_dims"]*4, args["dim_classification"])
            self.bn9 = nn.BatchNorm1d(  args["dim_classification"] )
            self.midClassification = nn.Linear(args["dim_classification"], args["dim_classification"])
            self.bn10 = nn.BatchNorm1d(  args["dim_classification"] )
            self.endClassification = nn.Linear(args["dim_classification"] , output_channels)

        else:
            self.classification= nn.Linear(args["final_dim_DGCNN"]*2, output_channels)


    
    def dgcnn_step(self,x):
        batch_size = x.size(0)
        x = get_graph_feature(x, k=self.k,device=self.device)           # (batch_size, 3, num_points) -> (batch_size, 3*2, num_points, k)
        x = self.conv1(x)                                               # (batch_size, 3*2, num_points, k) -> (batch_size, 64, num_points, k)
        x1 = x.max(dim=-1, keepdim=False)[0]                            # (batch_size, 64, num_points, k) -> (batch_size, 64, num_points)

        x = get_graph_feature(x1, k=self.k,device=self.device)          # (batch_size, 64, num_points) -> (batch_size, 64*2, num_points, k)
        x = self.conv2(x)                                               # (batch_size, 64*2, num_points, k) -> (batch_size, 64, num_points, k)
        x2 = x.max(dim=-1, keepdim=False)[0]                            # (batch_size, 64, num_points, k) -> (batch_size, 64, num_points)

        x = get_graph_feature(x2, k=self.k,device=self.device)          # (batch_size, 64, num_points) -> (batch_size, 64*2, num_points, k)
        x = self.conv3(x)                                               # (batch_size, 64*2, num_points, k) -> (batch_size, 128, num_points, k)
        x3 = x.max(dim=-1, keepdim=False)[0]                            # (batch_size, 128, num_points, k) -> (batch_size, 128, num_points)

        x = get_graph_feature(x3, k=self.k,device=self.device)          # (batch_size, 128, num_points) -> (batch_size, 128*2, num_points, k)
        x = self.conv4(x)                                               # (batch_size, 128*2, num_points, k) -> (batch_size, 256, num_points, k)
        x4 = x.max(dim=-1, keepdim=False)[0]                            # (batch_size, 256, num_points, k) -> (batch_size, 256, num_points)

        x = torch.cat((x1, x2, x3, x4), dim=1)                          # (batch_size, 64+64+128+256, num_points)

        x = self.conv5(x)                                               # (batch_size, 64+64+128+256, num_points) -> (batch_size, emb_dims, num_points)
        x1 = F.adaptive_max_pool1d(x, 1).view(batch_size, -1)           # (batch_size, emb_dims, num_points) -> (batch_size, emb_dims)
        x2 = F.adaptive_avg_pool1d(x, 1).view(batch_size, -1)           # (batch_size, emb_dims, num_points) -> (batch_size, emb_dims)
        x = torch.cat((x1, x2), 1)                                      # (batch_size, emb_dims*2)

        return x

    def forward(self, x,invert=False):
        if invert:
            first=x[:,1,:,:]
            second=x[:,0,:,:]
        else:
            first=x[:,0,:,:]
            second=x[:,1,:,:]

        #compute DGCNN output from both pointclouds 
        ret1=self.dgcnn_step(first)      
        ret2=self.dgcnn_step(second)

        #combine output and use classifier
        x=torch.cat((ret1,ret2),dim=1)
        if self.args["dim_classification"]>0:
            x=F.leaky_relu(self.bn9(self.beginClassification(x)), negative_slope=0.2)
            x=self.dp3(x)
            x=F.leaky_relu(self.bn10(self.midClassification(x)), negative_slope=0.2)
            x=self.dp3(x)
            x=self.endClassification(x)
        else: 
            x=self.classification(x)

        return x

## Training

In [43]:
#compute predictions and update loss and optimizer
def train_step(model,opt,criterion,data,label,normals,invert=False):
    
    data = data.permute(0,1,3,2)
    normals=normals.permute(0,1,3,2)
    opt.zero_grad()
    logits = model(data,normals,invert)
    logits=torch.squeeze(torch.sigmoid(logits))
    loss = criterion(logits, label)
    loss.backward()
    opt.step()
    preds=(logits>0.5).int()
    
    return preds,loss

In [44]:
#get some metrics and put in dictionary, useful for debug and logging with wandb
def get_metrics(train_true,train_pred,loss,epoch,suff="train"):
    
    acc=metrics.accuracy_score(train_true, train_pred)
    avg_acc= metrics.balanced_accuracy_score(train_true, train_pred)
    precision=metrics.precision_score(train_true,train_pred)
    recall=metrics.recall_score(train_true, train_pred)
    f1=metrics.f1_score(train_true,train_pred)

    d={f"{suff}_acc":acc,f"{suff}_avg_acc":avg_acc,f"{suff}_precision":precision,f"{suff}_recall":recall,
        f"{suff}_f1":f1,f"{suff}_loss":loss,"epoch":epoch}
    return d


In [45]:
#log weights of the layers, could lead to some interesting insights
def log_weights(model,epoch):
    weights=model.state_dict()
    log={}
    count=0
    for key in weights.keys():
        if "bn" not in key and ("weight" in key or "bias" in key):
            if "linear" in key.lower():
                log[key]=weights[key]
                
                mean=torch.mean(weights[key])
                var=torch.var(weights[key])
                std=torch.std(weights[key])
                log[f"{key}_mean"]=mean
                log[f"{key}_var"]=var
                log[f"{key}_std"]=std
    log["epoch"]=epoch
    return log

In [46]:

def train(args,extra_args, io):
    # num_workers=0 to make it work on windows
    train_loader = DataLoader(MyDataset(args,extra_args,partition='train'), 
            num_workers=extra_args["num_workers"],batch_size=args["batch_size"], shuffle=True, drop_last=True)
    test_loader = DataLoader(MyDataset(args,extra_args,partition='test'), 
            num_workers=extra_args["num_workers"],batch_size=args["test_batch_size"], shuffle=True, drop_last=False)

    device = torch.device("cuda" if args["cuda"] else "cpu")
    model = DGCNN_cls(args).to(device)

    print("Model loaded!")

    #not used, but useful if using multiple GPUs
    model = nn.DataParallel(model)
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    
    #choose optimizer
    if args["optimizer"].lower()=="sgd":
        print("Use SGD")
        opt = optim.SGD(model.parameters(), lr=args["lr"]*100,
                momentum=args["momentum"], weight_decay=args["weight_decay"])
    elif args["optimizer"].lower()=="adam":
        print("Use Adam")
        opt = optim.Adam(model.parameters(), lr=args["lr"], weight_decay=args["weight_decay"])

    #choose scheduler
    if args["scheduler"] == 'cos':
        scheduler = CosineAnnealingLR(opt, args["epochs"], eta_min=0,verbose=True)
    elif args["scheduler"] == 'step':
        scheduler = StepLR(opt, step_size=30, gamma=0.5,verbose=True)
    elif args["scheduler"].lower() =="reducelrplateau":
        scheduler = ReduceLROnPlateau(opt, 'min',factor=0.5,patience=3)
    
    criterion = cal_loss

    if extra_args["wand_project_name"]:
        if extra_args["wandb_resume"]:
            wandb.init(
                
                project=extra_args["wand_project_name"], 
                id=extra_args["wandb_resume"],
                resume="allow",
                entity="parmola",
                config=args)
        else:
            wandb.init(
                project=extra_args["wand_project_name"], 
                entity="parmola",
                config=args)

    start_epoch=0
    #load saved model
    if extra_args["load_path"]:
        checkpoint = torch.load(extra_args["load_path"])
        model.load_state_dict(checkpoint['model_state_dict'])
        scheduler.load_state_dict(checkpoint['scheduler'])
        opt.load_state_dict(checkpoint['optimizer_state_dict'])
        start_epoch = checkpoint['epoch']

    print(args)
    
    # TRAINING
    best_train_acc=0
    best_test_acc=0
    for epoch in range(start_epoch,args["epochs"]):
        train_loss = 0.0
        count = 0.0
        model.train()
        train_pred = []
        train_true = []
        print("Starting epoch ",epoch)
        for data,normals, label in train_loader:
            #print("get data")
            data, label = data.to(device), label.to(device).squeeze()
            normals=normals.to(device).float()
            data=data.float()
            label=label.float()
            batch_size = data.size()[0]
            
            count += batch_size
            preds,loss=train_step(model,opt,criterion,data,label,normals,invert=False)
            train_loss += loss.item() * batch_size
            train_true.append(label.cpu().numpy())
            train_pred.append(preds.detach().cpu().numpy())
            
            #train also using inverted pairs of the point clouds
            if extra_args["inverted_pair"]:
                count += batch_size
                preds,loss=train_step(model,opt,criterion,data,label,invert=True)
                train_loss += loss.item() * batch_size
                train_true.append(label.cpu().numpy())
                train_pred.append(preds.detach().cpu().numpy())
            
        
        if args["scheduler"] == 'cos':
            scheduler.step()
        elif args["scheduler"] == 'step':
            if opt.param_groups[0]['lr'] > 1e-5:
                scheduler.step()
            if opt.param_groups[0]['lr'] < 1e-5:
                for param_group in opt.param_groups:
                    param_group['lr'] = 1e-5        
        elif args["scheduler"].lower() =="reducelrplateau":
            scheduler.step(train_loss*1.0/count)
        try:
            actual_lr=scheduler.get_last_lr()
        except:
            actual_lr=opt.param_groups[0]['lr']
        if extra_args["wand_project_name"]:
            wandb.log({"actual_lr": actual_lr,"epoch":epoch})

        train_true = np.concatenate(train_true)
        train_pred = np.concatenate(train_pred)
        
        scores=get_metrics(train_true,train_pred,train_loss*1.0/count,epoch,"train")
        
        outstr = 'Train %d, loss: %.6f, train acc: %.6f, f1: %.6f' % (epoch,train_loss*1.0/count,
                                                                        scores["train_acc"],
                                                                        scores["train_f1"])
        io.cprint(outstr)
        

        if extra_args["wand_project_name"]:
            wandb.log(scores)
            if  scores["train_acc"]>best_train_acc:
                best_train_acc= scores["train_acc"]
                wandb.log({"best_train_acc": best_train_acc,"epoch":epoch})
                
            weights_log=log_weights(model,epoch)
            #wandb.log(weights_log)

        ####################
        # Test
        ####################
        test_loss = 0.0
        count = 0.0
        model.eval()
        test_pred = []
        test_true = []
        for data,normals, label in test_loader:
            data, label = data.to(device), label.to(device).squeeze()
            data=data.float()
            label=label.float()
            normals=normals.to(device).float()
            batch_size = data.size()[0]
            data = data.permute(0,1,3,2)
            logits = model(data,normals)
            logits=torch.squeeze(torch.sigmoid(logits))
            loss = criterion(logits, label)
            preds=(logits>0.5).int()
    

            count += batch_size
            test_loss += loss.item() * batch_size
            test_true.append(label.cpu().numpy())
            test_pred.append(preds.detach().cpu().numpy())
        
        test_true = np.concatenate(test_true)
        test_pred = np.concatenate(test_pred)

        scores=get_metrics(test_true,test_pred,test_loss*1.0/count,epoch,"test")
        
        outstr = 'Test %d, loss: %.6f, test acc: %.6f, test f1: %.6f' %(epoch,test_loss*1.0/count,
                                                                        scores["test_acc"],
                                                                        scores["test_f1"])
        
        io.cprint(outstr)
        if extra_args["wand_project_name"]:
            wandb.log(scores)
            if scores["test_acc"]>best_test_acc:
                best_test_acc=scores["test_acc"]
                wandb.log({"best_test_acc": best_test_acc,"epoch":epoch})
        
        if epoch % extra_args["saving_step"]==0:
            file_path=f"model_{epoch}of{args['epochs']}epoch_{args['info']}.pt"
            file_path=os.path.join("/kaggle/working",file_path)
            
            f=open(file_path,'w')
            
            #save model and optimizer
            torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': opt.state_dict(),
            'scheduler': scheduler.state_dict(),
            }, file_path)
            if extra_args["wand_project_name"]:
                wandb.save(file_path)
             
        #end epoch
        
    
    if extra_args["wand_project_name"]: wandb.finish()



In [47]:

def test(args,extra_args, io):
        test_loader = DataLoader(MyDataset(args,extra_args,partition='val'), 
                num_workers=extra_args["num_workers"],batch_size=args["batch_size"], shuffle=True, drop_last=True)

        device = torch.device("cuda" if args["cuda"] else "cpu")

        model = DGCNN_cls(args).to(device)

        model = nn.DataParallel(model)
       
        model.load_state_dict(torch.load(args["model_path"]))
        model = model.eval()
        test_acc = 0.0
        count = 0.0
        test_true = []
        test_pred = []
        for data,normals, label in test_loader:
                data, label = data.to(device), label.to(device).squeeze()
                data=data.float()
                label=label.float()
                normals=normals.to(device).float()
                batch_size = data.size()[0]
                data = data.permute(0,1,3,2)
                logits = model(data,normals)
                logits=torch.squeeze(torch.sigmoid(logits))
                preds=(logits>0.5).int()
                
                count += batch_size
                test_true.append(label.cpu().numpy())
                test_pred.append(preds.detach().cpu().numpy())

                #useless to log these
                test_loss=0
                epoch=args["epochs"]
                
        scores=get_metrics(test_true,test_pred,test_loss,epoch,"test")

        outstr = 'Test %d, loss: %.6f, test acc: %.6f, test f1: %.6f' %(epoch,test_loss*1.0/count,
                                                                        scores["test_acc"],
                                                                        scores["test_f1"])
        io.cprint(outstr)



## Main

In [ ]:

def _init_():
    if not os.path.exists('outputs'):
        os.makedirs('outputs')
    if not os.path.exists('outputs/'+extra_args["exp_name"]):
        os.makedirs('outputs/'+extra_args["exp_name"])
    if not os.path.exists('outputs/'+extra_args["exp_name"]+'/'+'models'):
        os.makedirs('outputs/'+extra_args["exp_name"]+'/'+'models')

if __name__ == "__main__":

    #stats I don't need to log
    extra_args={
        "exp_name": "exp",              
        "wand_project_name": "",
        "load_path":"",                     #path to load
        "wandb_resume":"",                  #id of the run to resume
        "dataset_file_path": "datasets\dataset_179608pairs_250points_center_random_normals_chunk_alpha1.hdf5",
        "eval": False,
        "saving_step":10,                   #how much epochs between savings
        "num_workers": 2,                   # parallel workers on dataloader, 0 if windows
        "inverted_pair": False,             #if true, the dataset use also the inverted pair of fragment for the training
    }

    #Stats I need to log
    args={
        #general
        "info": "new_dataset_best_parameters_new_lr",

        "num_train_samples": 2000,     # Num of pairs to consider for train
        "num_test_samples":1000,       # Num of pairs to consider for test
        "batch_size": 32,               # Size of batch
        "test_batch_size": 32,          # Size of batch
        "num_workers": 2,               # parallel workers on dataloader, 0 if windows

        #model
        "architecture": "DGCNN",    
        "num_points": 250,          # num of points to use
        "dropout": 0.3,             # initial dropout rate
        "point_dropout":0.2,
        "emb_dims": 1024,           # Dimension of embeddings
        "k": 30,                    # Num of nearest neighbors to use
        "final_dim_DGCNN": 512,     #dimension of final linear layer of DGCNN, max=256
        "dim_classification": 256,   #dim of mid final classification layer, max= final_dim_DGCNN*2

        #training
        "epochs": 50,               # number of episode to train
        "optimizer": "adam",        # optimizer [adam,sgd]
        "lr": 1e-4,                # learning rate (default: 0.001, 0.1 if using sgd)
        "momentum": 0.9,            # SGD momentum (default: 0.9)
        "scheduler": "reducelrplateau",         # Scheduler to use, [cos, step,reducelrplateau]
        "weight_decay": 1e-4,       # weight_decay for optimizer
        "seed": 1,                  # random seed (default: 1)
    }
    
    _init_()

    io = IOStream('outputs/' + extra_args["exp_name"] + '/run.log')
    io.cprint(str(args))
    
    #set up GPU
    args["cuda"] = torch.cuda.is_available()
    torch.manual_seed(args["seed"])
    if args["cuda"]:
        io.cprint(
            'Using GPU : ' + str(torch.cuda.current_device()) + ' from ' + str(torch.cuda.device_count()) + ' devices')
        torch.cuda.manual_seed(args["seed"])
    else:
        io.cprint('Using CPU')

    if not extra_args["eval"]:
        train(args,extra_args, io)
        wandb.finish()
        torch.cuda.empty_cache()
            
    else:
        test(args,extra_args, io)
